In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

In [2]:
# Implementation of Gradient Boost Regression
# Weak learners are small trees. Loss function is half of sum of squared error.
# Choice of loss reduces the gradient -dL(yi,F(xi))/dF(xi), F(x)=Fm-1(x) to a residual

class GradientBoostingRegressor:
    def __init__(self, learning_rate = 0.1, n_estimators = 100, max_depth_of_WL = 3):
        self.nu = learning_rate
        self.M = n_estimators
        self.max_depth = max_depth_of_WL
        self.F_array = []
        
    def fit(self,X,y):
        F0 = lambda X: np.array([y.mean()] * len(X))
        self.F_array.append(F0)
        for m in range(1,self.M):
            pseudo_residuals = y - self.predict(X)
            Fm = WeakLearner(X, pseudo_residuals, self.max_depth)
            self.F_array.append(Fm)
  
    def predict(self,X):
        m = len(self.F_array)
        predictions = self.F_array[0](X) + self.nu*sum(
                    self.F_array[i].predict(X) for i in range(1,m))
        return predictions

class WeakLearner:
    def __init__(self, X, error, max_depth = 3):
        self.clf = DecisionTreeRegressor(max_depth = max_depth)
        self.clf.fit(X, error)
    def predict(self, X):
        predictions = self.clf.predict(X)
        return predictions

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse

scaler = StandardScaler()
data = datasets.load_boston()
X,y = scaler.fit_transform(data.data), data.target
X_train, X_val, y_train, y_val = train_test_split(X,y)

In [4]:
def compute_mse(f):
    f.fit(X_train,y_train)
    mse_val = mse(y_val,f.predict(X_val))
    print("{:.1f}".format(mse_val))

In [5]:
myGBR = GradientBoostingRegressor()
print('My GBR')
compute_mse(myGBR)
from sklearn import ensemble

skRFC = ensemble.RandomForestRegressor()
print('sklearn RFC')
compute_mse(skRFC)

skGBR = ensemble.GradientBoostingRegressor(criterion = 'mse')
print('sklearn GBR')
compute_mse(skGBR)

My GBR
11.5
sklearn RFC
13.6
sklearn GBR
11.4


In [6]:
## Depending on the train/test split, the performance of the three models differ.
## However in general, my GBR has comparable performance to the sklearn GBR